In [93]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Decision Tree Regressor

In [94]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None


class DecisionTreeRegressor:
    def __init__(self, min_samples_split=2, max_depth=None, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth = max_depth if max_depth is not None else float("inf")
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)
        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_var_red = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the variance reduction
                curr_var_red = self.variance_reduction(y, X_column, thr)

                if curr_var_red > best_var_red:
                    best_var_red = curr_var_red
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def variance_reduction(self, y, X_column, threshold):
        parent=y
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        l_child=y[left_idxs]
        r_child=y[right_idxs]
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _most_common_label(self, y):
        value=np.mean(y)
        return value


    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
    def find_depth(self, tree):
        if tree is None: return 0
        if tree.left is None and tree.right is None:
            return 0
        return 1+max(self.find_depth(tree.left),self.find_depth(tree.right))
    
    def get_depth(self):
        return self.find_depth(self.root)-1
    

In [95]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.datasets import load_diabetes

# Load the diabetes dataset
diabetes_dataset = load_diabetes()

# Extract features and target variable
X = diabetes_dataset.data  # Features
y = diabetes_dataset.target 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

# Random Forest Regressor

In [96]:
class RandomForestRegressor:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_feature=None):
        self.n_trees = n_trees
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split
        self.n_features=n_feature
        self.trees = []
        self.tree_depths = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTreeRegressor(max_depth=self.max_depth,
                            min_samples_split=self.min_samples_split,
                            n_features=self.n_features)
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):
        return np.mean(y)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        predictions = np.array([self._most_common_label(pred) for pred in tree_preds])
        return predictions
    
    def calculate_depths(self):
        for t in self.trees:
            self.tree_depths.append(t.get_depth())
        return self.tree_depths    

# Train Model

In [105]:
reg = RandomForestRegressor(n_trees=100,max_depth=None)
reg.fit(X_train, y_train)

In [106]:
# Evaluating the model
from sklearn.metrics import mean_squared_error, r2_score
 
# Making predictions on the same data or new data
predictions = reg.predict(X_test)
 
# Evaluating the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')
 
r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')

Mean Squared Error: 3387.5532022471907
R-squared: 0.38820513400165446


In [107]:
print(reg.calculate_depths())

[14, 16, 15, 14, 15, 14, 14, 13, 13, 15, 17, 14, 14, 18, 14, 15, 16, 13, 15, 18, 15, 15, 13, 16, 18, 17, 13, 15, 14, 14, 19, 13, 17, 16, 17, 13, 14, 21, 15, 15, 17, 12, 14, 14, 15, 17, 13, 15, 15, 16, 15, 13, 15, 14, 14, 16, 13, 14, 13, 14, 15, 16, 14, 16, 15, 16, 15, 16, 16, 16, 16, 16, 13, 13, 17, 17, 15, 15, 16, 13, 15, 14, 15, 15, 14, 18, 13, 15, 16, 15, 13, 13, 15, 16, 16, 16, 17, 14, 15, 13]
